In [17]:
import requests
import pandas as pd
import json

# import eia_api.json
with open("eia_api.json", "r") as f:
    config = json.load(f)

# ---- 설정 ----
API_KEY = config["eia_api"]

url = "https://api.eia.gov/v2/petroleum/pri/spt/data/"
params = {
    "api_key": API_KEY,
    "frequency": "monthly",
    "data[0]": "value",
    "facets[product][]": "EPJK",  # Kerosene-type jet fuel
    "start": "2005-01",
    "end": "2025-05",
    "sort[0][column]": "period",
    "sort[0][direction]": "asc",   # ascending order for time series
    "offset": 0,
    "length": 5000
}

r = requests.get(url, params=params, timeout=30)
js = r.json()

# Extract into DataFrame
df = pd.DataFrame(js["response"]["data"])
df["date"] = pd.to_datetime(df["period"])
df = df[["date", "value"]].rename(columns={"value": "jet_fuel_usd_per_gal"})
df = df.sort_values("date").set_index("date")

print(df.head(), "\n", df.tail())
# df.to_csv("jet_fuel_price_2005_2025.csv", encoding="utf-8-sig")

df.to_csv("data/jet_fuel_price_2005_2025.csv", encoding="utf-8-sig")
print("Saved to data/jet_fuel_price_2005_2025.csv")


           jet_fuel_usd_per_gal
date                           
2005-01-01                1.334
2005-02-01                1.334
2005-03-01                1.562
2005-04-01                1.573
2005-05-01                1.471 
            jet_fuel_usd_per_gal
date                           
2024-12-01                2.097
2025-01-01                2.347
2025-02-01                2.264
2025-03-01                2.069
2025-04-01                1.976
Saved to data/jet_fuel_price_2005_2025.csv
